In [35]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Import dataset avec exclusion header & footer

In [36]:
df = pd.read_excel('../../data/DATABASE.xls', index_col=0, skiprows=[0,1,2], skipfooter=42)  

## Renommage colonnes

In [37]:
# Rename columns
df = df.rename(columns={'Sender c': 'Sender1', 
                        'Sender 2c': 'Sender2', 
                        'Sender 3c': 'Sender3', 
                        'Targetd': 'Target',
                        'US casee': 'US_case', 
                        'US unilateral casef': 'US_unilateral_case', 
                        'Foreign policy goal categoryg': 'Goal_categ', 
                        'First yearh': 'First_year',
                        'Last yeari': 'Last_year', 
                        'Policy resultj': 'Policy_result', 
                        'Sanctions contributionk': 'Sanctions_contrib',
                        'Success \nscorel': 'Success_score', 
                        'Companion policiesm': 'Comp_policies', 
                        'International cooperationn':'Int_coop', 
                        'International assistanceo': 'Int_assistance', 
                        'Cooperating international organizationp': 'Coop_int_org',
                        'International organization senderq': 'Int_org_sender', 
                        'International organization sender & target membersr': 'Int_org_sender_&_target_members', 
                        'Length (years)s': 'Length_years',
                        'Prior relationst': 'Prior_relations', 
                        'Regime Type \n(DEMOC 1st year)u': 'Regime_type', 
                        'Regime Type2 (Polity2 1st year)v': 'Regime_type2', 
                        'Regime Type3 \n(Scale 1st year)w': 'Regime_type3',
                        'Political stability priorx': 'Political_stability_prior', 
                        'Political stability duringy': 'Political_stability_during',
                        'Cost to targetz': 'Cost_to_target', 
                        'Cost to target (percent of GNP)aa': 'Cost_to_target_%GNP',
                        'Cost to target per capitabb': 'Cost_to_target_per_capita', 
                        'Trade linkagecc': 'Trade_linkage', 
                        'GNP \nratiodd': 'GNP_ratio',
                        'Health and stabilityee': 'Health_&_stability', 
                        'Sanction typeff': 'Sanction_type', 
                        'Cost to sendergg': 'Cost_to_sender',
                        'GDP growth (percent, 5-year average)hh': 'GDP_growth%5years_average', 
                        'Inflation (percent,\n 3-year average)ii':'Inflation%3years_average', 
                        'Target International Monetary Fund codejj': 'Target_int_monetary_code', 
                        'Country groupkk': 'Country_group'
                       })
# rename index
df.index.names = ['id_case']

In [38]:
#  list categ variables
list(df.select_dtypes(include=['object']).columns)

['Sender1',
 'Sender2',
 'Sender3',
 'Target',
 'Goal',
 'Last_year',
 'Comp_policies',
 'Int_assistance',
 'Coop_int_org',
 'Regime_type',
 'Regime_type2',
 'Regime_type3',
 'Political_stability_prior',
 'Political_stability_during',
 'Cost_to_target',
 'Cost_to_target_%GNP',
 'Cost_to_target_per_capita',
 'Trade_linkage',
 'Sanction_type',
 'GDP_growth%5years_average',
 'Inflation%3years_average',
 'Target_int_monetary_code']

In [39]:
#  list quantiv variables
# df_quant = df.select_dtypes(include=['int','float64'])
list(df.select_dtypes(include=['int','float64']).columns)
# df_quant.describe()

['GNP_ratio']

## Valeurs manquantes

In [40]:
# df.isna().sum()  

### Colones Cost to target - Cost to target (percent of GNP) - Cost to target per capita   
Remplacement des '--' en 0 et après converion en numérique

In [41]:
df[['Cost_to_target_%GNP', 'Cost_to_target', 'Cost_to_target_per_capita']].head()

,Cost_to_target_%GNP,Cost_to_target,Cost_to_target_per_capita
id_case,,,
14-1,7.1,843,12.58
17-1,0.8,23,0.44
18-1,4.1,446,2.49
21-1,--,--,--
25-1,--,--,--


In [42]:
df['Cost_to_target_%GNP'] = df['Cost_to_target_%GNP'].astype('str')
df['Cost_to_target_%GNP'] = df['Cost_to_target_%GNP'].str.replace('--', '0')
df['Cost_to_target_%GNP'] = pd.to_numeric(df['Cost_to_target_%GNP'])

df['Cost_to_target'] = df['Cost_to_target'].astype('str')
df['Cost_to_target'] = df['Cost_to_target'].str.replace('--', '0')
df['Cost_to_target'] = pd.to_numeric(df['Cost_to_target'])

df['Cost_to_target_per_capita'] = df['Cost_to_target_per_capita'].astype('str')
df['Cost_to_target_per_capita'] = df['Cost_to_target_per_capita'].str.replace('--', '0')
df['Cost_to_target_per_capita'] = pd.to_numeric(df['Cost_to_target_per_capita'])

In [43]:
df[['Cost_to_target_%GNP', 'Cost_to_target', 'Cost_to_target_per_capita']].head()

,Cost_to_target_%GNP,Cost_to_target,Cost_to_target_per_capita
id_case,,,
14-1,7.1,843.0,12.58
17-1,0.8,23.0,0.44
18-1,4.1,446.0,2.49
21-1,0.0,0.0,0.00
25-1,0.0,0.0,0.00


In [44]:
df[['Cost_to_target_%GNP', 'Cost_to_target', 'Cost_to_target_per_capita']].dtypes

Cost_to_target_%GNP          float64
Cost_to_target               float64
Cost_to_target_per_capita    float64
dtype: object

### Colonnes "GDP growth (percent, 5-year average) ", "Inflation": - remplacement des n.a. par NaN

In [45]:
df['GDP_growth%5years_average'] = df['GDP_growth%5years_average'].astype('str')
df['GDP_growth%5years_average'] = df['GDP_growth%5years_average'].str.replace('n.a.', '')
df['GDP_growth%5years_average'] = pd.to_numeric(df['GDP_growth%5years_average'])

df['GDP_growth%5years_average'].head()
# df['GDP_growth%5years_average'].dtypes
# df['GDP_growth%5years_average'].isna().sum()

id_case
14-1    3.58
17-1    5.40
18-1     NaN
21-1   -1.50
25-1    3.30
Name: GDP_growth%5years_average, dtype: float64

### Colonne "Comp_policies"
Suppression de la virgule  
'--' restent tels quels

In [46]:
df['Comp_policies'] = df['Comp_policies'].str.replace(',', '')
df['Comp_policies'].head()

id_case
14-1     R
17-1    --
18-1    RQ
21-1    --
25-1    --
Name: Comp_policies, dtype: object

In [49]:
df.head()

,Sender1,Sender2,Sender3,Target,Goal,US_case,US_unilateral_case,Goal_categ,First_year,Last_year,Policy_result,Sanctions_contrib,Success_score,Comp_policies,Int_coop,Int_assistance,Coop_int_org,Int_org_sender,Int_org_sender_&_target_members,Length_years,Prior_relations,Regime_type,Regime_type2,Regime_type3,Political_stability_prior,Political_stability_during,Cost_to_target,Cost_to_target_%GNP,Cost_to_target_per_capita,Trade_linkage,GNP_ratio,Health_&_stability,Sanction_type,Cost_to_sender,GDP_growth%5years_average,Inflation%3years_average,Target_int_monetary_code,Country_group
id_case,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,1918,4,3,12,R,4,A,--,0,0,4,1,5,2,2,0,0,843.0,7.1,12.58,9,1.0,3,"F,X,M",4,3.58,2.86667,134,1
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,1918,2,2,4,--,1,--,--,0,0,1,2,5,1,2,0,0,23.0,0.8,0.44,20.5,13.0,3,X,2,5.40,25.6,158,1
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1920,1,2,2,RQ,4,--,--,0,0,2,1,1,-1,2,0.1,0,446.0,4.1,2.49,18.5,1.0,1,"F,X,M",3,NaN,n.a.,922,2
21-1,LEAGUE OF NATIONS,NaN,NaN,YUGOSLAVIA,Military disruption vs. Albania,0,0,3,1921,1921,4,4,16,--,4,--,--,1,1,1,2,3,0,2,0.4,0,0.0,0.0,0.00,26.5,37.0,2,--,2,-1.50,n.a.,188,2
25-1,LEAGUE OF NATIONS,NaN,NaN,GREECE,Withdraw from Bulgaria,0,0,3,1925,1925,4,4,16,--,4,--,--,1,1,1,2,0,-6,1,0.9,0,0.0,0.0,0.00,36,56.0,2,--,2,3.30,48.8,174,1


## Modelling part

In [75]:
# Defining X as all variables different from 'Success score', 'Policy result' and 'sanction contriution' and dropping 
# the last two as they are correlated with the first one.

data_table = df.drop(['Policy_result', 'Sanctions_contrib'], axis=1)

# Copying 'Success score' column then dropping the first (to have it as last column)
data_table['Success score'] = data_table['Success_score']
data_table = data_table.drop(['Success_score'], axis=1)

In [76]:
# Replacing remaining inconsistent values ('--' and 'n.a.') for MLP classifier
data_table = data_table.replace(to_replace='--', value= None)
data_table = data_table.replace(to_replace='n.a.', value= None)

In [77]:
data_table.head()

,Sender1,Sender2,Sender3,Target,Goal,US_case,US_unilateral_case,Goal_categ,First_year,Last_year,Comp_policies,Int_coop,Int_assistance,Coop_int_org,Int_org_sender,Int_org_sender_&_target_members,Length_years,Prior_relations,Regime_type,Regime_type2,Regime_type3,Political_stability_prior,Political_stability_during,Cost_to_target,Cost_to_target_%GNP,Cost_to_target_per_capita,Trade_linkage,GNP_ratio,Health_&_stability,Sanction_type,Cost_to_sender,GDP_growth%5years_average,Inflation%3years_average,Target_int_monetary_code,Country_group,Success score
id_case,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,1918,R,4,A,--,0,0,4,1,5,2,2,0,0,843.0,7.1,12.58,9,1.0,3,"F,X,M",4,3.58,2.86667,134,1,12
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,1918,R,1,A,--,0,0,1,2,5,1,2,0,0,23.0,0.8,0.44,20.5,13.0,3,X,2,5.40,25.6,158,1,4
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1920,RQ,4,A,--,0,0,2,1,1,-1,2,0.1,0,446.0,4.1,2.49,18.5,1.0,1,"F,X,M",3,NaN,25.6,922,2,2
21-1,LEAGUE OF NATIONS,NaN,NaN,YUGOSLAVIA,Military disruption vs. Albania,0,0,3,1921,1921,RQ,4,A,--,1,1,1,2,3,0,2,0.4,0,0.0,0.0,0.00,26.5,37.0,2,--,2,-1.50,25.6,188,2,16
25-1,LEAGUE OF NATIONS,NaN,NaN,GREECE,Withdraw from Bulgaria,0,0,3,1925,1925,RQ,4,A,--,1,1,1,2,0,-6,1,0.9,0,0.0,0.0,0.00,36,56.0,2,--,2,3.30,48.8,174,1,16


In [70]:
#data_table['GDP_growth%5years_average'] = lb_make.fit_transform(data_table['GDP_growth%5years_average'])

In [78]:
# Dropping 'Sender 2/3' columns as they have lot of NaNs
data_table = data_table.drop(['Sender2','Sender3'], axis =1)

In [80]:
data_table.head()

,Sender1,Target,Goal,US_case,US_unilateral_case,Goal_categ,First_year,Last_year,Comp_policies,Int_coop,Int_assistance,Coop_int_org,Int_org_sender,Int_org_sender_&_target_members,Length_years,Prior_relations,Regime_type,Regime_type2,Regime_type3,Political_stability_prior,Political_stability_during,Cost_to_target,Cost_to_target_%GNP,Cost_to_target_per_capita,Trade_linkage,GNP_ratio,Health_&_stability,Sanction_type,Cost_to_sender,GDP_growth%5years_average,Inflation%3years_average,Target_int_monetary_code,Country_group,Success score
id_case,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,GERMANY,Military victory,0,0,4,1914,1918,R,4,A,--,0,0,4,1,5,2,2,0,0,843.0,7.1,12.58,9,1.0,3,"F,X,M",4,3.58,2.86667,134,1,12
17-1,UNITED STATES,JAPAN,Shipping for Allies,1,1,5,1917,1918,R,1,A,--,0,0,1,2,5,1,2,0,0,23.0,0.8,0.44,20.5,13.0,3,X,2,5.40,25.6,158,1,4
18-1,UNITED KINGDOM,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1920,RQ,4,A,--,0,0,2,1,1,-1,2,0.1,0,446.0,4.1,2.49,18.5,1.0,1,"F,X,M",3,NaN,25.6,922,2,2
21-1,LEAGUE OF NATIONS,YUGOSLAVIA,Military disruption vs. Albania,0,0,3,1921,1921,RQ,4,A,--,1,1,1,2,3,0,2,0.4,0,0.0,0.0,0.00,26.5,37.0,2,--,2,-1.50,25.6,188,2,16
25-1,LEAGUE OF NATIONS,GREECE,Withdraw from Bulgaria,0,0,3,1925,1925,RQ,4,A,--,1,1,1,2,0,-6,1,0.9,0,0.0,0.0,0.00,36,56.0,2,--,2,3.30,48.8,174,1,16


#### Encoding categorical variables to numeric ones

In [81]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
lb_make = LabelEncoder()

In [82]:
# Transforming categorical values to numeeric values
for c in data_table.columns:
    if(data_table[c].dtype == 'O'):
        data_table[c] = lb_make.fit_transform(data_table[c])
        #print(type(c))   

In [83]:
data_table.head()

,Sender1,Target,Goal,US_case,US_unilateral_case,Goal_categ,First_year,Last_year,Comp_policies,Int_coop,Int_assistance,Coop_int_org,Int_org_sender,Int_org_sender_&_target_members,Length_years,Prior_relations,Regime_type,Regime_type2,Regime_type3,Political_stability_prior,Political_stability_during,Cost_to_target,Cost_to_target_%GNP,Cost_to_target_per_capita,Trade_linkage,GNP_ratio,Health_&_stability,Sanction_type,Cost_to_sender,GDP_growth%5years_average,Inflation%3years_average,Target_int_monetary_code,Country_group,Success score
id_case,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,21,43,105,0,0,4,1914,0,4,4,0,0,0,0,4,1,7,13,1,0,0,843.0,7.1,12.58,27,1.0,3,4,4,3.58,27,3,1,12
17-1,23,60,130,1,1,5,1917,0,4,1,0,0,0,0,1,2,7,12,1,0,0,23.0,0.8,0.44,50,13.0,3,6,2,5.40,113,7,1,4
18-1,21,96,37,0,0,2,1918,1,5,4,0,0,0,0,2,1,3,10,1,1,0,446.0,4.1,2.49,45,1.0,1,4,3,NaN,113,87,2,2
21-1,13,122,98,0,0,3,1921,2,5,4,0,0,1,1,1,2,5,11,1,4,0,0.0,0.0,0.00,61,37.0,2,0,2,-1.50,113,13,2,16
25-1,13,45,139,0,0,3,1925,3,5,4,0,0,1,1,1,2,2,5,0,8,0,0.0,0.0,0.00,73,56.0,2,0,2,3.30,126,9,1,16


In [84]:
data_table.dtypes

Sender1                              int32
Target                               int32
Goal                                 int32
US_case                              int64
US_unilateral_case                   int64
Goal_categ                           int64
First_year                           int64
Last_year                            int32
Comp_policies                        int32
Int_coop                             int64
Int_assistance                       int32
Coop_int_org                         int32
Int_org_sender                       int64
Int_org_sender_&_target_members      int64
Length_years                         int64
Prior_relations                      int64
Regime_type                          int32
Regime_type2                         int32
Regime_type3                         int32
Political_stability_prior            int32
Political_stability_during           int32
Cost_to_target                     float64
Cost_to_target_%GNP                float64
Cost_to_tar

#### Training part with NN model (supervised)

In [128]:
#data_table = data_table.replace(to_replace="NaN", value=None)
data_table = data_table.fillna(0)

In [129]:
X, y = data_table.iloc[:,:33], data_table['Success score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [131]:
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

print("Accuracy on training set : {:.2f}".format(mlp.score(X_train, y_train)))
print("Accuracy on test set : {:.2f}".format(mlp.score(X_test, y_test)))

Accuracy on training set : 0.40
Accuracy on test set : 0.22
